In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 

# Load and preprocess the dataset (use FER-2013 or similar facial expression datasets)
def load_dataset():
    # Assuming FER-2013 dataset or similar format with "pixels" and "emotion" columns
    import pandas as pd
    data = pd.read_csv('fer2013.csv')  # Ensure file exists in the working directory
    X = []
    y = []
    for i in range(len(data)):
        pixels = np.array(data['pixels'][i].split(), dtype='float32').reshape(48, 48)
        X.append(pixels)
        y.append(data['emotion'][i])
    X = np.array(X)
    y = np.array(y)
    X = X / 255.0  # Normalize pixel values to [0,1] range for better training stability  # Normalize pixel values
    y = to_categorical(y)  # One-hot encode emotion labels
    return X, y



# Build the CNN model
def build_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')  # Assuming 7 emotion classes
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate the model
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = build_model((48, 48, 1))
    model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_test, y_test))
    return model, X_test, y_test

# Real-time emotion detection with webcam
import os

def emotion_detection(model):
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    cap = cv2.VideoCapture(0)

    # Try loading the Haarcascade file
    haarcascade_path = "haarcascade_frontalface_default.xml"
    if not os.path.exists(haarcascade_path):
        raise ValueError(f"Cannot find {haarcascade_path}. Ensure it is in the same directory as the script.")

    face_cascade = cv2.CascadeClassifier(haarcascade_path)

    if face_cascade.empty():
        raise ValueError(f"Error loading Haarcascade from {haarcascade_path}. Check if the file is corrupted.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        detected_faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in detected_faces:
            roi = gray[y:y+h, x:x+w]
            roi = cv2.resize(roi, (48, 48))
            roi = roi.reshape(1, 48, 48, 1) / 255.0
            prediction = model.predict(roi)
            emotion = emotion_labels[np.argmax(prediction)]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow('Emotion Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# Main function
if __name__ == "__main__":
    X, y = load_dataset()
    X = X.reshape(-1, 48, 48, 1)  # Reshape for CNN
    model, X_test, y_test = train_model(X, y)
    print("Model trained successfully. Starting real-time emotion detection.")
    emotion_detection(model)




C:\Users\kudik\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 54s 108ms/step - accuracy: 0.2637 - loss: 1.8047 - val_accuracy: 0.3991 - val_loss: 1.5964
Epoch 2/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 82s 109ms/step - accuracy: 0.3806 - loss: 1.5828 - val_accuracy: 0.4384 - val_loss: 1.4797
Epoch 3/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 80s 106ms/step - accuracy: 0.4161 - loss: 1.5030 - val_accuracy: 0.4503 - val_loss: 1.4177
Epoch 4/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 50s 112ms/step - accuracy: 0.4432 - loss: 1.4375 - val_accuracy: 0.4819 - val_loss: 1.3449
Epoch 5/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 49s 108ms/step - accuracy: 0.4677 - loss: 1.3924 - val_accuracy: 0.4955 - val_loss: 1.3248
Epoch 6/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 49s 109ms/step - accuracy: 0.4770 - loss: 1.3552 - val_accuracy: 0.5032 - val_loss: 1.3070
Epoch 7/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 82s 108ms/step - accuracy: 0.4921 - loss: 1.3253 - val_accuracy: 0.5176 - val_loss: 1.2734
Epoch 8/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 82s 108ms/step - accuracy: 0.4889 - loss: 1